<a href="https://colab.research.google.com/github/sjanorkar/LLM-finetuning/blob/main/llm_finetuning_falcon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install bitsandbytes==0.41.3
!pip3 install peft==0.11.1
!pip3 install trl==0.8.6
!pip3 install accelerate==0.30.1
!pip3 install datasets==2.19.2
!pip3 install transformers==4.41.2


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 34.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-many

In [2]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from peft import (LoraConfig, get_peft_model, get_peft_model_state_dict)
from datasets import load_dataset
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM

In [3]:
dataset_name = "Vezora/Tested-22k-Python-Alpaca"
model_id = "tiiuae/falcon-rw-1b"

# Load model

In [4]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16)


model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map={"": 0})

model.config.use_cache = False
model.config.pretraining_tp = 1


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.62G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

# Load Tokenizer

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

# Pre fine tuning inference

In [17]:
%%time
inputs = tokenizer('''Write a function to find the maximum difference between two prime numbers in a given array. The array can contain positive and negative integers, and can be unsorted. Additionally, the function should handle arrays of any length. The function should return the maximum difference as an absolute value. For example, for the array [5, 3, 17, 11, 9], the function should return 14. However, your function should have a time complexity of O(n), where n is the length of the array. Additionally, you should not use any built-in functions or libraries to check if a number is prime. You need to implement your own prime checking function.''', return_tensors="pt", return_attention_mask=False)

outputs = model.generate(**inputs, max_length=300, temperature=0.5)
text = tokenizer.batch_decode(outputs)[0]
print(text)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1659: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') befor

Write a function to find the maximum difference between two prime numbers in a given array. The array can contain positive and negative integers, and can be unsorted. Additionally, the function should handle arrays of any length. The function should return the maximum difference as an absolute value. For example, for the array [5, 3, 17, 11, 9], the function should return 14. However, your function should have a time complexity of O(n), where n is the length of the array. Additionally, you should not use any built-in functions or libraries to check if a number is prime. You need to implement your own prime checking function.
The following is a sample input file:
Input: [5, 3, 17, 11, 9] Output: 14
The following is a sample output file:
Output: 14
The following is a sample input file:
Input: [5, 3, 17, 11, 9] Output: 14
The following is a sample output file:
Output: 14
The following is a sample input file:
Input: [5, 3, 17, 11, 9] Output: 14
The following is a sample output file:
Output

In [18]:
%%time
inputs = tokenizer('''Write a python code to Output the factorial of 7 using a recursive function and without using any loops.''', return_tensors="pt", return_attention_mask=False)

outputs = model.generate(**inputs, max_length=300, temperature=0.5)
text = tokenizer.batch_decode(outputs)[0]
print(text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Write a python code to Output the factorial of 7 using a recursive function and without using any loops.
Write a python code to Output the factorial of 7 using a recursive function and without using any loops.
Write a python code to Output the factorial of 7 using a recursive function and without using any loops.
Write a python code to Output the factorial of 7 using a recursive function and without using any loops.
Write a python code to Output the factorial of 7 using a recursive function and without using any loops.
Write a python code to Output the factorial of 7 using a recursive function and without using any loops.
Write a python code to Output the factorial of 7 using a recursive function and without using any loops.
Write a python code to Output the factorial of 7 using a recursive function and without using any loops.
Write a python code to Output the factorial of 7 using a recursive function and without using any loops.
Write a python code to Output the factorial of 7 using 

In [19]:
%%time
inputs = tokenizer('''Write a python code to return true if number is even else return false''', return_tensors="pt", return_attention_mask=False)

outputs = model.generate(**inputs, max_length=300, temperature=0.5)
text = tokenizer.batch_decode(outputs)[0]
print(text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Write a python code to return true if number is even else return false
Write a python code to return true if number is even else return false
Write a python code to return true if number is even else return false
Write a python code to return true if number is even else return false
Write a python code to return true if number is even else return false
Write a python code to return true if number is even else return false
Write a python code to return true if number is even else return false
Write a python code to return true if number is even else return false
Write a python code to return true if number is even else return false
Write a python code to return true if number is even else return false
Write a python code to return true if number is even else return false
Write a python code to return true if number is even else return false
Write a python code to return true if number is even else return false
Write a python code to return true if number is even else return false
Write 

# PEFT parameters

In [20]:
LORA_R = 16
LORA_ALPHA = 16
LORA_DROPOUT= 0.05
LORA_TARGET_MODULES = [
    "q_proj",
    "k_proj",
    "v_proj",
    "o_proj",
    "gate_proj",
    "up_proj",
    "down_proj",
    "lm_head",
]

BATCH_SIZE = 64
MICRO_BATCH_SIZE = 4
GRADIENT_ACCUMULATION_STEPS = BATCH_SIZE // MICRO_BATCH_SIZE
LEARNING_RATE = 3e-4
TRAIN_STEPS = 300
OUTPUT_DIR = "/home/finetuned/falcon-rw-1b/Tested-22k-Python-Alpaca"

In [21]:
peft_params = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=LORA_TARGET_MODULES,
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM"
    )

# Training params

In [22]:
training_params = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    save_steps=100,
    logging_steps=100,
    learning_rate=2e-4,
    evaluation_strategy="steps",
    weight_decay=0.001,
    fp16=False,
    bf16=True,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


# Load Dataset

In [23]:
dataset = load_dataset(dataset_name)
instruction = dataset["train"]["instruction"]
instruction

input = dataset["train"]["input"]
output = dataset["train"]["output"]

import datasets
from datasets import Dataset
temp_dataset_1 = Dataset.from_dict({"instruction": instruction[0:20000], "input": input[0:20000], "output": output[0:20000]})
dataset_1 = datasets.DatasetDict({"train": temp_dataset_1})
dataset_1

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 20000
    })
})

In [24]:
def generate_prompt(data_point):
    return f"""Write a python code for following problem statement
### Instruction:
{data_point["instruction"]}
### Response:
{data_point["output"]}"""

CUTOFF_LEN = 3056
def tokenize(prompt, add_eos_token=True):
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=CUTOFF_LEN,
        padding=False,
        return_tensors=None,
    )
    if (
        result["input_ids"][-1] != tokenizer.eos_token_id
        and len(result["input_ids"]) < CUTOFF_LEN
        and add_eos_token
    ):
        result["input_ids"].append(tokenizer.eos_token_id)
        result["attention_mask"].append(1)

    result["labels"] = result["input_ids"].copy()

    return result

def generate_and_tokenize_prompt(data_point):
    full_prompt = generate_prompt(data_point)
    tokenized_full_prompt = tokenize(full_prompt)
    return tokenized_full_prompt

In [25]:
train_val = dataset_1["train"].train_test_split(
    test_size=100, shuffle=True, seed=42
)
train_data = (
    train_val["train"].map(generate_and_tokenize_prompt)
)
val_data = (
    train_val["test"].map(generate_and_tokenize_prompt)
)

Map:   0%|          | 0/19900 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [27]:
model = get_peft_model(model, peft_params)
model.print_trainable_parameters()


trainable params: 837,632 || all params: 1,312,462,848 || trainable%: 0.0638


In [28]:
data_collator = transformers.DataCollatorForSeq2Seq(
    tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True,
)

In [ ]:
%%time
trainer = transformers.Trainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=val_data,
    args=training_params,
    data_collator=data_collator
)
model.config.use_cache = False
old_state_dict = model.state_dict
model.state_dict = (
    lambda self, *_, **__: get_peft_model_state_dict(
        self, old_state_dict()
    )
).__get__(model, type(model))

# Clear cache to free up memory
torch.cuda.empty_cache()

model = torch.compile(model)

trainer.train()
model.save_pretrained(OUTPUT_DIR)

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Step,Training Loss,Validation Loss
100,1.460800,1.361184
200,1.334100,1.278148
300,1.273300,1.215240
400,1.195700,1.165117
500,1.165100,1.129391
600,1.135100,1.101799
700,1.110400,1.077039
800,1.084700,1.058782
900,1.076500,1.046335
1000,1.065700,1.035170


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:180: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:180: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:180: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:180: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers fou

In [86]:
%%time
inputs = tokenizer('''Write a python code for following problem statement, return the factorial of 7 using a recursive function and without using any loops.''', return_tensors="pt", return_attention_mask=False)

outputs = model.generate(**inputs, max_length=100, temperature=0.3)
text = tokenizer.batch_decode(outputs)[0]
print(text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Write a python code for following problem statement, return the factorial of 7 using a recursive function and without using any loops.
### Instruction:
1. The factorial of a number is the number of times the number is divided by itself.
2. The factorial of a number is the number of times the number is divided by itself.
3. The factorial of a number is the number of times the number is divided by itself.
4. The factorial of a
CPU times: user 3.43 s, sys: 5.06 ms, total: 3.43 s
Wall time: 3.37 s


In [79]:
%%time
inputs = tokenizer('''Write a python code for following problem statement, \nfind the maximum difference between two prime numbers in a given array. The array can contain positive and negative integers, and can be unsorted. Additionally, the function should handle arrays of any length. The function should return the maximum difference as an absolute value. For example, for the array [5, 3, 17, 11, 9], the function should return 14. However, your function should have a time complexity of O(n), where n is the length of the array. Additionally, you should not use any built-in functions or libraries to check if a number is prime. You need to implement your own prime checking function.''', return_tensors="pt", return_attention_mask=False)

outputs = model.generate(**inputs, max_length=300)
text = tokenizer.batch_decode(outputs)[0]
print(text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Write a python code for following problem statement, 
find the maximum difference between two prime numbers in a given array. The array can contain positive and negative integers, and can be unsorted. Additionally, the function should handle arrays of any length. The function should return the maximum difference as an absolute value. For example, for the array [5, 3, 17, 11, 9], the function should return 14. However, your function should have a time complexity of O(n), where n is the length of the array. Additionally, you should not use any built-in functions or libraries to check if a number is prime. You need to implement your own prime checking function. Additionally, you should implement a function to check if a number is prime or not. Additionally, you should implement a function to check if a number is prime or not. Additionally, you should implement a function to check if a number is prime or not. Additionally, you should implement a function to check if a number is prime or no

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
inputs = tokenizer('''Write a function to find the maximum difference between two prime numbers in a given array. The array can contain positive and negative integers, and can be unsorted. Additionally, the function should handle arrays of any length. The function should return the maximum difference as an absolute value. For example, for the array [5, 3, 17, 11, 9], the function should return 14. However, your function should have a time complexity of O(n), where n is the length of the array. Additionally, you should not use any built-in functions or libraries to check if a number is prime. You need to implement your own prime checking function.''', return_tensors="pt", return_attention_mask=False)

outputs = model.generate(**inputs, max_length=300)
text = tokenizer.batch_decode(outputs)[0]
print(text)

In [ ]:
model.push_to_hub("swapnilj/falcon-rw-1b-sj")